In [ ]:
import ROOT

In [ ]:
# Open Data
## Dataset triggered by a electron/photon trigger
# f_data = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataEgamma.root")
f_data = ROOT.TFile.Open("/trisep/collider/DataEgamma.root")

## Dataset triggered by a muon trigger
# f_data = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataMuons.root")
# f_data = ROOT.TFile.Open("/trisep/collider/DataMuons.root")

# Load Tree, named mini
tree = f_data.Get("mini")
# Load entries and print number of events
tree.GetEntries()

In [ ]:
# Define histograms

# TH1F constructor takes "name" (must be unique), "Title" (can be repeated), x-axis label, y-axis label, nBins, min, max
h_lep_pt_all = ROOT.TH1F("lep_pt_all", "All lepton pT; pT [GeV]; events",25,0,100)
h_lep_pt_2lep = ROOT.TH1F("lep_pt_2lep", "lepton pT in events w/ 2+ leptons; pT [GeV]; events",25,0,100)

In [ ]:
# Main loop that runs over all events, does selections, fills histograms

# Keep track of number of events I've run over
nEvents = 0
nEventsToRun = 100000 # useful to test on a smaller number

# Declare TLorentzVectors to be used later
leadLepton  = ROOT.TLorentzVector()
trailLepton = ROOT.TLorentzVector()

# Loop over events

for event in tree:

    nEvents += 1

    ## print number of events, useful to make sure something is happening 
    if(nEvents%10000==0):
        print("nEvents = ",nEvents)
    if(nEvents==nEventsToRun):
        break
        
    ## looping over all leptons in event
    for j in range(tree.lep_n):

        # Fill lepton histograms. Divide by 1000 to get GeV from MeV
        h_lep_pt_all.Fill(tree.lep_pt[j]/1000)


    # Cut #1: At least 2 leptons in event
    if tree.lep_n >= 2:
            
        # Set each TLorentzVector equal to leading and trailing lepton
        # Constructor of TLorentzVector takes pT, eta, phi, E (or other forms, see ROOT documentation)
        leadLepton.SetPtEtaPhiE(tree.lep_pt[0]/1000., tree.lep_eta[0], tree.lep_phi[0], tree.lep_E[0]/1000.)
        trailLepton.SetPtEtaPhiE(tree.lep_pt[1]/1000., tree.lep_eta[1], tree.lep_phi[1], tree.lep_E[1]/1000.)

        # Technically this only plots the first two leptons in the event, in the case that there are more
        h_lep_pt_2lep.Fill(leadLepton.Pt())
        h_lep_pt_2lep.Fill(trailLepton.Pt())
        
        # Can add two TLorentz vectors together to get sum
        Zcandidate = leadLepton + trailLepton

        # Fill more histograms about Zcandidate 
        # ie, Zcandidate.M() gives invariant mass
        # For more accessors, see: https://root.cern.ch/doc/master/classTLorentzVector.html
                

In [ ]:
# Make plots!

# First, define canvas
c1 = ROOT.TCanvas("Canvas","first canvas",800,600)

# Draw histograms

h_lep_pt_all.SetFillColor(3)
h_lep_pt_all.Draw()
c1.Draw()
c1.Print("electron_pT_all.pdf")

In [ ]:
# Compare shapes of lepton pT distributions: all leptons in all events, 
    # to leading and subleading leptons in 2 lepton events

c2 = ROOT.TCanvas("Canvas 2","Lepton pTs",800,600)

h_lep_pt_all.SetFillStyle(3002)
h_lep_pt_all.SetFillColor(3)
h_lep_pt_all.SetLineColor(3)

h_lep_pt_2lep.SetFillStyle(3003)
h_lep_pt_2lep.SetFillColor(4)
h_lep_pt_2lep.SetLineColor(4)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h_lep_pt_all,"all Leptons ","l")
legend.AddEntry(h_lep_pt_2lep,"Leading 2 leptons","l")

h_lep_pt_all.SetStats(0)
h_lep_pt_2lep.SetStats(0)

# To compare shapes, draw distributions normalized to unity area
h_lep_pt_all.DrawNormalized()
h_lep_pt_2lep.DrawNormalized("same")

legend.Draw()
c2.Draw()